In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 57, in main
    service.run()
  File "

In [3]:
# Install Required Libraries:

!pip install -q \
  torch torchvision \
  transformers==4.50.3 \
  datasets==3.5.0 \
  evaluate==0.4.3 \
  matplotlib \
  pillow==11.1.0

Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.11/dist-packages/rapids_dask_dependency/dask_loader.py", line 6, in <module>
    import importlib.util
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 980, in exec_module
  File "<frozen importlib.util>", line 18, in <module>
  File "/usr/lib/python3.11/threading.py", line 1539, in <module>
    _main_thread = _MainTh

In [4]:
# 2. Imports and Setup

from PIL import Image
from pathlib import Path
import random
import torch
import numpy as np
import evaluate
import logging
import matplotlib.pyplot as plt
from datasets import Dataset, DatasetDict
from transformers import (
    ViTImageProcessor,
    ViTForImageClassification,
    TrainingArguments,
    Trainer
)

from google.colab import drive, files
import os

# Enable logging
logging.basicConfig(
    filename="training_log.txt",
    filemode='w',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [ ]:
# 3. Download & Unzip Dataset

!pip install -q gdown

import gdown
import zipfile

# Download from Google Drive
file_id = "1RbGjWoB0OAXK0vWJo1ggBtfWS8f8uwSq"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="datasetTWO.zip", quiet=False)

# Unzip dataset
with zipfile.ZipFile("datasetTWO.zip", 'r') as zip_ref:
    zip_ref.extractall("datasetTWO")


Downloading...
From (original): https://drive.google.com/uc?id=1RbGjWoB0OAXK0vWJo1ggBtfWS8f8uwSq
From (redirected): https://drive.google.com/uc?id=1RbGjWoB0OAXK0vWJo1ggBtfWS8f8uwSq&confirm=t&uuid=e1c99dce-1a39-4441-b0f7-863ca2347222
To: /content/datasetTWO.zip
100%|██████████| 69.3M/69.3M [00:00<00:00, 107MB/s]


In [ ]:
# 4. Load Dataset

from pathlib import Path
from PIL import Image
import random

def load_dataset_split(split_folder, max_per_class=200):
    data = []
    for label_name, label in [("real", 0), ("fake", 1)]:
        folder_path = Path(split_folder) / label_name
        image_paths = list(folder_path.rglob("*"))
        image_paths = [p for p in image_paths if p.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".webp"]]
        !ls
        print(image_paths)
        selected_paths = random.sample(image_paths, min(max_per_class, len(image_paths)))

        for p in selected_paths:
            try:
                img = Image.open(p).convert("RGB")
                data.append({"image": img, "label": label})
            except Exception as e:
                print(f"Could not load image {p}: {e}")
    return data

train_data = load_dataset_split("datasetTWO/wild_dataset/train", max_per_class=420)
val_data   = load_dataset_split("datasetTWO/wild_dataset/valid", max_per_class=90)
test_data  = load_dataset_split("datasetTWO/wild_dataset/test", max_per_class=90)

from datasets import Dataset, DatasetDict

dataset = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(val_data),
    "test": Dataset.from_list(test_data),
})

dataset     datasetTWO.zip  loss_curve.png  vit-deepfake-finetune
datasetTWO  dataset.zip     sample_data
[PosixPath('datasetTWO/wild_dataset/train/real/415_3071.png'), PosixPath('datasetTWO/wild_dataset/train/real/28_740.png'), PosixPath('datasetTWO/wild_dataset/train/real/48_444.png'), PosixPath('datasetTWO/wild_dataset/train/real/253_1922.png'), PosixPath('datasetTWO/wild_dataset/train/real/40_737.png'), PosixPath('datasetTWO/wild_dataset/train/real/103_2279.png'), PosixPath('datasetTWO/wild_dataset/train/real/83_681.png'), PosixPath('datasetTWO/wild_dataset/train/real/77_2117.png'), PosixPath('datasetTWO/wild_dataset/train/real/288_3184.png'), PosixPath('datasetTWO/wild_dataset/train/real/232_5971.png'), PosixPath('datasetTWO/wild_dataset/train/real/37_1374.png'), PosixPath('datasetTWO/wild_dataset/train/real/41_1456.png'), PosixPath('datasetTWO/wild_dataset/train/real/107_2437.png'), PosixPath('datasetTWO/wild_dataset/train/real/593_4520.png'), PosixPath('datasetTWO/wild_dataset/t

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch
import numpy as np

# 5. Prepare Model & Trainer

image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

def transform(example_batch):
    inputs = image_processor([img for img in example_batch["image"]], return_tensors="pt")
    inputs["labels"] = example_batch["label"]
    return inputs

dataset["train"] = dataset["train"].with_transform(transform)
dataset["validation"] = dataset["validation"].with_transform(transform)

def collate_fn(batch):
    return {
        "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
        "labels": torch.tensor([x["labels"] for x in batch])
    }

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=2,
    id2label={"0": "real", "1": "fake"},
    label2id={"real": 0, "fake": 1}
)

# Improved hyperparameters
training_args = TrainingArguments(
   output_dir="./vit-deepfake-finetune",
   per_device_train_batch_size=32,           # Smaller for regularization
   gradient_accumulation_steps=2,            # Same effective batch size (64)
   fp16=True,
   num_train_epochs=12,                      # A bit more training time
   learning_rate=2e-5,                       # Lower = more stable convergence
   weight_decay=0.05,                        # Stronger regularization
   warmup_ratio=0.1,                         # Longer ramp-up
   lr_scheduler_type="cosine",               # Smooth decaying LR
   label_smoothing_factor=0.1,               # Helps generalization
   logging_steps=10,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   save_total_limit=2,
   remove_unused_columns=False,
   report_to="none",
   load_best_model_at_end=True,
   metric_for_best_model="eval_f1",          # F1 is better for class balance
   greater_is_better=True
)


# returns F1 and others
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted")
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=image_processor,
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-61-44da5ee49f71>:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [5]:
import torch

if torch.cuda.is_available():
    print("GPU is active:", torch.cuda.get_device_name(0))
else:
    print("GPU not available — double check runtime settings.")


GPU not available — double check runtime settings.


In [ ]:
# 6. Train & Save Model

trainer.train()

model.save_pretrained("./vit-deepfake-finetune/best_model")
image_processor.save_pretrained("./vit-deepfake-finetune/best_model")


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.695700,0.685423,0.561111,0.549415,0.568191,0.561111
2,0.643500,0.671371,0.588889,0.567532,0.610769,0.588889
3,0.620000,0.647940,0.688889,0.687500,0.692308,0.688889
4,0.541200,0.614501,0.694444,0.693982,0.695628,0.694444
5,0.444400,0.583583,0.722222,0.721672,0.723992,0.722222
6,0.421300,0.562569,0.755556,0.753086,0.766204,0.755556
7,0.355200,0.540185,0.750000,0.748689,0.755327,0.750000
8,0.316500,0.528778,0.766667,0.765944,0.770000,0.766667
9,0.285400,0.525400,0.777778,0.777531,0.779018,0.777778
10,0.272900,0.524470,0.794444,0.793674,0.798910,0.794444


['./vit-deepfake-finetune/best_model/preprocessor_config.json']

In [ ]:
# 7. Evaluate on Test Set

dataset["test"] = dataset["test"].with_transform(transform)
metrics = trainer.evaluate(eval_dataset=dataset["test"])
test_acc = metrics["eval_accuracy"]

logging.info(f"Test Set Accuracy: {test_acc:.4f}")
logging.info("=== Training Complete ===")
print("Test Set Accuracy:", test_acc)

Test Set Accuracy: 0.8166666666666667


In [6]:
# 8. Plot Loss Curve

train_loss = [log["loss"] for log in trainer.state.log_history if "loss" in log]
eval_loss = [log["eval_loss"] for log in trainer.state.log_history if "eval_loss" in log]
steps_train = [log["step"] for log in trainer.state.log_history if "loss" in log]
steps_eval = [log["step"] for log in trainer.state.log_history if "eval_loss" in log]

plt.figure(figsize=(10, 6))
plt.plot(steps_train, train_loss, label="Training Loss")
plt.plot(steps_eval, eval_loss, label="Validation Loss")
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("loss_curve.png")
plt.show()

NameError: name 'trainer' is not defined